# KoBART를 활용한 도서 요약 데이터셋 학습 및 추론


[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

이번 실습에서는 사전학습된 한국어 언어모델인 KoBART를 활용하여 도서 문서를 요약하는 요약모델을 학습해봅시다!

## 1. dataset download

AI Hub의 도서 요약 데이터셋을 활용합니다.   
전처리가 완료된 데이터셋은 [링크](https://drive.google.com/file/d/1SBKsb9hy9vmOm5Cfi0GDOrpm-_kqXOyS/view?usp=share_link)에서 받으실 수 있으며,

해당 데이터셋에 대한 이용 약관은 [여기](https://www.aihub.or.kr/useStplat.do?currMenu=110&topMenu=110)에 명시되어 있습니다.

In [ ]:
data = open('/content/drive/MyDrive/summary/modi_output.txt', 'r', encoding='utf-8')
lines = data.readlines()
passages = []
summaries = []

for line in lines:
    line = line.strip()
    if len(line.split('\t')) < 2:
        print(line)
        break
    passages.append('<s>' + line.split('\t')[0] + '</s>')
    # kobart 학습시에는 앞 뒤로 '<s>'와 '</s>'를 삽입해야 잘 나옵니다.. 왜그런지 몰?루..
    summaries.append('<s>' + line.split('\t')[1] + '</s>')
    # 주의! 이렇게 끝에 </s>만 붙여서 tokenizing하시면 나중에 max_len에 의해 </s>가 truncation될 수 있습니다!
    # 요약을 위한 모델이기 때문에, passage가 model의 max_len을 넘어가는 경우엔 학습 데이터에서 제거하는 것이 좋습니다.

data.close()

In [ ]:
print("passages:", passages[0])
print("summaries:", summaries[0])

passages: <s>자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다. 따라서 생각을 명료하게 정리하는 것이 명확한 글쓰기의 시작이다. 존 케네스 갈브레이드(John Kenneth Galbraith) 교수는 “아무리 복잡하고 어려운 주제라고 명쾌한 언어로 설명이 가능하다. 하지만 필자가 그 주제에 대해 완전히 이해하지 못하면 절대로 명확하게 쓸 수 없다.”고 했다. 명료한 글은 명료한 생각에서 나온다. 그렇지만 글을 고치다보면 생각도 명료하게 된다. 명료하게 쓰려면 내용을 단순화해야 한다. 많은 것을 전달하려는 욕심을 버리고 한두가지로 선택하고 거기 집중해야 한다. 앞에서 이야기한 간결성은 명확성에 도움이 된다. 명료함은 복잡한 설득기법을 사용하는 것보다 더 중요하다. 판사는 이해할 수 없는 사실관계 진술로는 설득되지 않는다. 설득기법이 명료함을 해치는 때에는 그 기법을 사용하지 않는다.</s>
summaries: <s>주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 명확한 글쓰기는 당연 가능하다. 명확한 글쓰기를 할 때 가장 중요한 점은 내용의 단순화 및 간결성이고 많은 것을 전달하려는 욕심을 버리는 것이다.</s>


In [ ]:
print(len(passages))

160002


## 2. make dataset

Huggingface에서 데이터를 쉽게 다루기 위해 datasets로 묶어줍니다.

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    passages[0:1000],   # 16만 데이터를 모두 사용하면, 학습 시간이 1 epoch에 약 2시간 30분 정도 소요됩니다.
    summaries[0:1000],  # 그래서 실습에서는 1000개만 사용하겠습니다.
    test_size=0.2,
    shuffle=True
    )

In [ ]:
print(len(x_train))
print(len(x_test))

800
200


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
dataset = DatasetDict(
    {
        'train': Dataset.from_dict({'passage':x_train, 'summary':y_train}),
        'valid': Dataset.from_dict({'passage':x_test, 'summary':y_test}),
    }
)

In [ ]:
print(dataset['train'][0])

{'passage': '<s>생활지도 문항 중에서 부모 합산하여 점수가 가장 높은 문항은 아이가 공공 장소에서 다른 사람에게 피해주는 일이 없도록 신경 씀(4.1점), 아이의 긍정적인 행동에 대해 충분히 칭찬함(4.1점)으로 나타났다. 이는 모두 실행역량 문항에 포함되는 것으로 인식, 성장역량 문항에 비해서 점수가 높았다. 전체적으로 문항별 점수가 크게 다르지 않으나 통계적으로 유의한 4개의 문항에 대해 부의 점수가 모에 비해 0.15점 낮은 것으로 나타났다. 또한 다른 영역과 달리 생활지도 영역의 실행역량의 경우, 부모 간의 역량 차이가 크지 않은 것으로 나타났으나 인식과 성장역량에서 다른 영역과 마찬가지로 부모 간 역량 차이가 나 부에 비해 모의 역량이 더 높은 것으로 확인되었다.</s>', 'summary': '<s>생활지도 영역에서는 실행역량의 두 문항이 부모 합산 점수가 가장 높았다. 문항별 점수 중 통계적으로 유의한 4개 문항이나 생활지도 영역의 인식, 성장역량은 모의 점수가 부보다 높게 나타났다.</s>'}


## 3. tokenizing datasets

먼저 kobart-base-v2의 tokenizer를 다운받습니다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")

In [ ]:
print(tokenizer.special_tokens_map)

{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


In [ ]:
tokenizer.tokenize('<s>')

['<s>']

In [ ]:
sents = ["이순신은 조선 중기의 무신이다.", "이순신 짱"]

In [ ]:
tokenized_sents = tokenizer(sents)

In [ ]:
for keys in tokenized_sents.keys():
    print(keys, ':', tokenized_sents[keys]) # 길이가 다릅니다

input_ids : [[22577, 19293, 15015, 14059, 15525, 14113, 11467, 16247], [22577, 11467, 1700, 12371]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]


tokenizing 옵션을 줄 수 있습니다.

In [ ]:
tokenized_sents = tokenizer(sents, max_length=6, truncation=True, padding=True)
for keys in tokenized_sents.keys():
    print(keys, ':', tokenized_sents[keys])

input_ids : [[22577, 19293, 15015, 14059, 15525, 14113], [22577, 11467, 1700, 12371, 3, 3]]
token_type_ids : [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
attention_mask : [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0]]


전체 데이터셋을 tokenizing하여 저장하겠습니다.

In [ ]:
max_input_length = 512
max_target_length = 512

def preprocess_function(examples):
    inputs = [doc for doc in examples["passage"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)  # 단, padding은 주지 않습니다.
    # padding은 data_collator가 만들어줄겁니다.

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():   # kobart의 경우, 동일한 tokenizer를 사용합니다.
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocessed_dataset_sample = preprocess_function(dataset['train'][0:1])

for keys in preprocessed_dataset_sample.keys():
    print(keys, ':', preprocessed_dataset_sample[keys])

input_ids : [[0, 14923, 14931, 14111, 13603, 18609, 15714, 14350, 11211, 14330, 14244, 15026, 14377, 14904, 14111, 13603, 12005, 19736, 15641, 14137, 18290, 14355, 19977, 14783, 14785, 15029, 22382, 17219, 1700, 11667, 15672, 15275, 12136, 14505, 22497, 20316, 29211, 14225, 17050, 20624, 13599, 15672, 15275, 12136, 18247, 19743, 14130, 15002, 14422, 19349, 11803, 10256, 14111, 13603, 11786, 14740, 14343, 14173, 16083, 243, 14966, 11803, 10256, 14111, 13603, 11786, 14085, 14668, 14244, 15026, 14276, 18332, 26335, 14111, 13603, 10872, 14244, 15026, 15003, 27386, 14076, 14665, 19672, 14157, 23900, 13590, 14136, 15957, 14111, 13603, 11786, 14225, 27104, 14244, 15026, 14069, 11786, 15685, 27305, 252, 12136, 16867, 14173, 19743, 14130, 14638, 14355, 17601, 9120, 15616, 14923, 14931, 17601, 12024, 19349, 11803, 18306, 18543, 15714, 18406, 23563, 17828, 24107, 14687, 14173, 19743, 14665, 16083, 9120, 14966, 11803, 10256, 14030, 14355, 17601, 9120, 19177, 15714, 14313, 23563, 17828, 14054, 1406

In [ ]:
print(tokenizer.decode(preprocessed_dataset_sample['input_ids'][0]))

<s> 생활지도 문항 중에서 부모 합산하여 점수가 가장 높은 문항은 아이가 공공 장소에서 다른 사람에게 피해주는 일이 없도록 신경 씀(4.1점), 아이의 긍정적인 행동에 대해 충분히 칭찬함(4.1점)으로 나타났다. 이는 모두 실행역량 문항에 포함되는 것으로 인식, 성장역량 문항에 비해서 점수가 높았다. 전체적으로 문항별 점수가 크게 다르지 않으나 통계적으로 유의한 4개의 문항에 대해 부의 점수가 모에 비해 0.15점 낮은 것으로 나타났다. 또한 다른 영역과 달리 생활지도 영역의 실행역량의 경우, 부모 간의 역량 차이가 크지 않은 것으로 나타났으나 인식과 성장역량에서 다른 영역과 마찬가지로 부모 간 역량 차이가 나 부에 비해 모의 역량이 더 높은 것으로 확인되었다.</s>


In [ ]:
print(tokenizer.decode(preprocessed_dataset_sample['labels'][0]))

<s> 생활지도 영역에서는 실행역량의 두 문항이 부모 합산 점수가 가장 높았다. 문항별 점수 중 통계적으로 유의한 4개 문항이나 생활지도 영역의 인식, 성장역량은 모의 점수가 부보다 높게 나타났다.</s>


동작을 확인했으니, 학습 데이터를 tokenizing해서 저장하겠습니다.

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
for keys in tokenized_datasets['train'][0].keys():
    print(keys, ':', tokenized_datasets['train'][0][keys])

passage : <s>생활지도 문항 중에서 부모 합산하여 점수가 가장 높은 문항은 아이가 공공 장소에서 다른 사람에게 피해주는 일이 없도록 신경 씀(4.1점), 아이의 긍정적인 행동에 대해 충분히 칭찬함(4.1점)으로 나타났다. 이는 모두 실행역량 문항에 포함되는 것으로 인식, 성장역량 문항에 비해서 점수가 높았다. 전체적으로 문항별 점수가 크게 다르지 않으나 통계적으로 유의한 4개의 문항에 대해 부의 점수가 모에 비해 0.15점 낮은 것으로 나타났다. 또한 다른 영역과 달리 생활지도 영역의 실행역량의 경우, 부모 간의 역량 차이가 크지 않은 것으로 나타났으나 인식과 성장역량에서 다른 영역과 마찬가지로 부모 간 역량 차이가 나 부에 비해 모의 역량이 더 높은 것으로 확인되었다.</s>
summary : <s>생활지도 영역에서는 실행역량의 두 문항이 부모 합산 점수가 가장 높았다. 문항별 점수 중 통계적으로 유의한 4개 문항이나 생활지도 영역의 인식, 성장역량은 모의 점수가 부보다 높게 나타났다.</s>
input_ids : [0, 14923, 14931, 14111, 13603, 18609, 15714, 14350, 11211, 14330, 14244, 15026, 14377, 14904, 14111, 13603, 12005, 19736, 15641, 14137, 18290, 14355, 19977, 14783, 14785, 15029, 22382, 17219, 1700, 11667, 15672, 15275, 12136, 14505, 22497, 20316, 29211, 14225, 17050, 20624, 13599, 15672, 15275, 12136, 18247, 19743, 14130, 15002, 14422, 19349, 11803, 10256, 14111, 13603, 11786, 14740, 14343, 14173, 16083, 243, 14966, 11803, 10256, 14111, 13603, 11786, 14085, 14668, 14244

## 4. Fine-tuning using KoBART

이제 본격적으로 kobart 모델을 불러와 학습을 해보겠습니다.

In [ ]:
from transformers import BartForConditionalGeneration

In [ ]:
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

모델 구조를 확인해봅니다.

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir = "model_output",
    evaluation_strategy = "steps",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_total_limit = 2,
    num_train_epochs = 2,
    predict_with_generate = True,
    save_steps = 1000,
    eval_steps = 500
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py
# Data collator that will dynamically pad the inputs received, as well as the labels.

이제 evaluation을 위한 metric을 넣어보겠습니다.

In [ ]:
!pip install rouge_score

In [ ]:
from datasets import load_metric

metric = load_metric("rouge")

In [ ]:
!pip install kss==2.1.0

In [ ]:
import kss
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(kss.split_sentences(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(kss.split_sentences(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

Saving model checkpoint to model_output
Configuration saved in model_output/config.json
Model weights saved in model_output/pytorch_model.bin
tokenizer config file saved in model_output/tokenizer_config.json
Special tokens file saved in model_output/special_tokens_map.json


In [ ]:
!cp /content/model_output/pytorch_model.bin /content/drive/MyDrive/summary/.
!cp /content/model_output/config.json /content/drive/MyDrive/summary/.

## 5. Inference

이제 학습된 모델로 inference를 날려보겠습니다.

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained('MrBananaHuman/kobart-base-v2-summarization').to('cuda')
model.eval()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gogamza/kobart-base-v2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/54a37e9385f90886428b084042f151c1a699203416d41765d94aac4cddb5fd5c.d098ef3866c1da94bdfaa5c1f24ecb7c5c16b37423b79263fbd3668d2ae61f91
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

storing https://huggingface.co/MrBananaHuman/kobart-base-v2-summarization/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/2a767070f7e98353903ccb1ac5bfb4080195079cbb731542a8f0a22c55512b70.d4626372fd5133c852a84193d02699c123aa2f4700d3ed2d1cb7ec98570fa7bd
creating metadata file for /root/.cache/huggingface/transformers/2a767070f7e98353903ccb1ac5bfb4080195079cbb731542a8f0a22c55512b70.d4626372fd5133c852a84193d02699c123aa2f4700d3ed2d1cb7ec98570fa7bd
loading configuration file https://huggingface.co/MrBananaHuman/kobart-base-v2-summarization/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2a767070f7e98353903ccb1ac5bfb4080195079cbb731542a8f0a22c55512b70.d4626372fd5133c852a84193d02699c123aa2f4700d3ed2d1cb7ec98570fa7bd
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartFo

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

storing https://huggingface.co/MrBananaHuman/kobart-base-v2-summarization/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8f6bd1db12121bd5dba9e20202216e9d29f7c21ec822a2abe7194d4374d23674.86be1e574dbb785d7fdbe6a0d3a19060a1f9d82acc7d00ab197cb10b3ed3d293
creating metadata file for /root/.cache/huggingface/transformers/8f6bd1db12121bd5dba9e20202216e9d29f7c21ec822a2abe7194d4374d23674.86be1e574dbb785d7fdbe6a0d3a19060a1f9d82acc7d00ab197cb10b3ed3d293
loading weights file https://huggingface.co/MrBananaHuman/kobart-base-v2-summarization/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8f6bd1db12121bd5dba9e20202216e9d29f7c21ec822a2abe7194d4374d23674.86be1e574dbb785d7fdbe6a0d3a19060a1f9d82acc7d00ab197cb10b3ed3d293
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at MrBananaHuman/kobart-base-v2-summariza

디코딩 전략에 대해선 아래 링크를 참고해주세요    
https://colab.research.google.com/drive/1yUGVmQ0nj8Hd3h0YV6PemQx0FtzpefGB?usp=sharing

In [ ]:
import torch

def summarization(text):
    tokenized_text = tokenizer(text, return_tensors='pt', truncation=False).to('cuda')
    with torch.no_grad():
        outputs = model.generate(
            tokenized_text['input_ids'],
            do_sample = True,
            eos_token_id = tokenizer.eos_token_id,
            max_length = 512,
            # top_p = 0.7,
            # top_k = 20,
            num_beams=20,
            num_return_sequences = 1,
            no_repeat_ngram_size = 2,
            early_stopping = True
        )
        return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
summarization("""<s>[서울=뉴시스] 임종명 기자 = 러시아가 제안한 우크라이나 인도주의 상황에 관한 결의안이 유엔 안전보장이사회(안보리)를 통과하지 못했다.

AP통신과 CNN 등 외신은 23일(현지시간) 미국 뉴욕 유엔본부에서 진행된 안보리에서 미국을 포함한 13개국이 표결을 기원함으로써 러시아의 제안이 부결됐다고 보도했다.

결의안이 통과되려면 찬성 9표가 필요했는데, 이날 표결은 찬성 2표, 반대 0표, 기권 13표였다. 찬성 2표는 러시아와 중국이었던 것으로 전해졌다.

린다 토머스-그린필드 유엔주재 미국 대사는 투표에 앞서 우크라이나를 침공한 러시아가 다시 한번 안보리를 이용해 그들의 잔인한 행동을 희석시키려 하고 있다"고 말했다.

그는 "러시아가 국제사회에 우크라이나의 인도주의적 위기를 해결해 달라고 요청하는 결의를 내놓는 것은 정말 비양심적인 일'이라며 러시아는 악화하는 인도적 조건이나 수백만명의 생명과 꿈이 파괴된 전쟁에는 개의치 않는다. 만약 그들이 신경 쓰고 있다면 그들은 침공을 멈췄을 것"이라고 강조했다.

이 결의안에는 우크라이나에서 벌어지는 인도주의적 위기 해결을 위한 지원 등의 내용이 담겼으나 정작 그 위기를 만든 러시아의 침공에 대해서는 언급되지 않은 것으로 전해졌다.

토머스-그린필드 대사는 "이번 위기의 유일한 원인인 러시아의 역할에 대해서는 언급하지 않았다"며 "우리의 투표는 우리가 그 일에 관여하지 않을 것임을 보여줄 것"이라고 강조했다.

그러나 바실리 네벤자 주유엔 러시아 대사는 프랑스와 멕시코 등 다른 나라가 낸 결의안을 '러시아를 비난하기 위한 정치적 쇼'라고 주장하면서도 자국이 제출한 인도적 결의안이 다른 국가가 낸 안과 유사하다고 주장했다.

현재 유엔 총회에는 우크라이나 관련 결의안이 2건 상정돼 있다. 유엔은 오는 24일 총회를 통해 결의안에 대한 표결을 진행할 예정이다.</s>""")

'러시아가 제안한 우크라이나 인도주의 상황에 관한 결의안이 유엔 안전보장이사회(안보리)를 통과하지 못했다. 유엔주재 미국 대사는 결의안에서 인도주의적 위기 해결을 위한 지원 등의 내용이 담겼으나 정작 그 위기를 만든 러시아의 침공에 대해서는 언급되지 않은 것으로 전해졌다. 유엔은 오는 24일 총회를 통해 결의안에 대한 표결을 진행할 예정이다.'

In [ ]:
summarization("""<s>군집분석(cluster analysis)은 패턴인식, 기계학습 분야에서 자주 활용 되는 다변량 자료분석 알고리즘 중 하나이며 마케팅 분야에서 고객 세분화, 소매업분야에서 새로운 상품 또는 서비스 개발 등 실제 세계의 문제를 푸는데 널리 활용되는 방법론이다.
군집분석의 목적은 그룹의 수(clusters)를 정하고 같은 그룹에 할당된 관측치들은 유사한 성질(homogenous)을 갖도록 하고, 반면에 서로 다른 그룹에 할당된 관측치들은 이질적인 성질(heterogeneous)을 갖도록 하는 것이다.
군집분석은 다음과 같은 두 가지 사항에 대한 명시적인 답이 존재하지 않아 대표적인 비지도학습(unsupervised learning)의 문제로 간주된다.
첫째로 “주어진 데이터에 최적인 군집의 수는 몇 개인가?”라는 질문이며 둘째로는 “데이터의 개별 관측치의 최적의 군집 할당이 존재하는가?”라는 질문이다.
위의 두 가지에 질문에 대한 명시적인 답이 없기 때문에 군집분석의 결과를 평가하는 것은 어려운 문제이며 이를 해결하기 위해 다양한 군집 타당성 지표(cluster validity index)가 제안되었다(Halkidi et al., 2002; Halkidi et al., 2002; Liu et al., 2010; Halkidi et al., 2001).
이들은 “적절하게 수행된 군집분석의 결과물의 경우, 같은 군집에 할당된 관측치들은 동질적이고 서로 다른 군집에 할당된 관측치들은 이질적이다.”라는 개념을 공유하지만 실제로 군집 타당성 지표들은 위의 성질들을 각기 다른 수식으로 표현하여 결과를 산출한다.
하지만 어떤 지표도 모든 군집분석에 대해서 최적의 결과를 판단하지는 못한다고 알려져 있다</s>""")

'군집분석은 다변량 자료분석 알고리즘 중 하나이며 마케팅 분야에서 고객 세분화, 소매업분야에서 새로운 상품 또는 서비스 개발 등 실제 세계의 문제를 푸는데 널리 활용되는 방법론이다.'

In [ ]:
summarization("""<s>내비게이션 기술은 다양한 종류의 센서를 이용해 주변 장소에 대한 정보들을 획득하고, 이를 바탕으로 특정 목적을 수행하기 위한 이동 좌표를 결정하는 기술을 말하며, 자율주행 자동차, 탐사로봇 그리고 재난 구조 로봇 등, 많은 로봇 분야에서 응용되고 있다.
하지만, 기존의 내비게이션 기술은 미탐사 지역이나 복잡하고 새로운 지역에서 적용하기 어렵다는 단점이 있다.
이러한 한계점을 극복하기 위해, 본 연구에서는 동물의 뇌에서 일어나는 장소정보처리 원리를 이용한 새로운 내비게이션 모델을 구축하고자 한다.
동물의 해마체(hippocampal formation)은 장소정보를 처리하는데 특화된 영역이다.
특히, 환경의 경계면에 대한 정보를 처리하는 경계면 세포(boundary vector cell), 동물의 머리방향에 대한 정보를 처리하는 머리방향 세포(head direction cell), 공간의 격자 좌표 정보를 처리하는 격자 세포(grid cell), 그리고 특정 장소에 대한 정보를 처리하는 장소 세포(place cell)가 발견되었다.
이러한 장소정보처리 세포는 해마체 내에서 장소정보처리 신경회로를 통해 신경신호를 공유하며 장소정보를 처리하게 된다.
본 연구는, 뇌 장소정보처리 신경세포의 형태학적 구조와 전기생리학적 특성을 정교하게 모사할 수 있는 뉴로모픽 단일신경세포 모델을 구축하고, 해부학적으로 알려진 장소정보처리 신경회로를 기반으로 뉴로모픽 신경망 모델을 구축하는 것을 목적으로 한다.
또한, 구축된 뉴로모픽 신경망 모델을 가상 쥐 행동 시뮬레이터에 연결하여, 실제 쥐처럼 공간에서의 내비게이션을 수행할 수 있는지 연구하였다.
나아가, 쥐의 행동을 모사하는 쥐 로봇을 개발하고, 뉴로모픽 신경망 모델이 실제 환경에서 로봇을 통해 내비게이션을 수행하는지 관찰하였다.</s>""")

'내비게이션 기술은 다양한 센서를 이용해 주변 장소에 대한 정보를 획득하고, 이를 바탕으로 특정 목적을 수행하기 위한 이동 좌표를 결정하는 기술을 말하며, 많은 로봇 분야에서 응용되고 있으나, 기존의 기술은 미탐사 지역이나 복잡하고 새로운 지역에서 적용하기 어렵다는 단점이 있다.'

In [ ]:
summarization("""<s>인공지능 기술의 발전과 함께 기술을 적극적으로 활용하는 대표적인 분야 중 하나는 자연어처리 분야이다.
특히 BERT 등의 대규모 데이터로 사전 학습한 언어 모델의 발달로 획기적인 발전을 이루었으며, 많은 자연어처리 태스크에서 괄목할만한 성과를 이루었다.
그 중 질의응답 태스크로 알려진 기계 독해 문제 또한 대규모 데이터로 사전 학습한 모델로 인하여 문제 해결 능력 면에서 성능이 좋아졌으며, 특히 특정 분야에 특화된 질의응답 시스템은 챗봇 등의 애플리케이션으로 상용화되고 있다.
이러한 질의 응답시스템은 전문적인 내용을 포함하고 있으며 보다 정밀한 설계를 필요로 한다.
특정 도메인을 위한 질의응답 시스템이 원활히 작동하려면 대답에 확신이 없는 경우 질문을 회피하는 능력이 중요하다.
이러한 회피 능력에 대한 성능은 기존 연구에서 제시된 바 없어 본 연구는 실험을 통해 이를 밝히고자 한다.
특정 도메인을 위한 질의응답 시스템의 학습 방법은 범용 데이터로 학습된 모델에 도메인 데이터를 추가로 학습시키는 방법과 처음부터 도메인 데이터만으로 학습시키는 방법으로 나뉜다.
연구 결과 전자의 방법으로 학습한 모델이 전반적으로 뛰어난 회피 능력을 보였다.
또한, 특정 도메인 데이터의 사용 없이 마스킹 전략만을 바꾼 사전 학습 모델의 회피 성능이 도메인 데이터를 사용한 모델과 비슷하거나 상회한다는 사실을 확인했다.</s>""")

'인공지능어처리 분야는 대규모 데이터로 사전 학습한 언어 모델의 발달로 획기적인 발전을 이루었으며, 기계 독해 문제 또한 문제 해결 능력 면에서 성능이 좋아졌으며, 특히 특정 분야에 특화된 질의응답 시스템은 챗봇 등의 애플리케이션으로 상용화되고 있다.'